# Homework 2: Word Similarity

Student Name: Zihang Su

Student ID: 710118

Python version used: python 3

## General info

<b>Due date</b>: 11pm, Sunday April 1st

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 5% of mark for class

<b>Overview</b>: In this homework, you'll be quantifying the similarity between pairs of words using the structure of WordNet and word co-occurrence in the Brown corpus, using PMI, LSA, and word2vec. You will quantify how well these methods work by comparing to a carefully filtered human annotated gold-standard.

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> It is recommended to use Python 2 but we accept Python 3 solutions</b>. Make sure you state which version you used in the beggining of this notebook.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Extra credit</b>: Each homework has a task which is optional with respect to getting full marks on the assignment, but that can be used to offset any points lost on this or any other homework assignment (but not the final project or the exam). We recommend you skip over this step on your first pass, and come back if you have time: the amount of effort required to receive full marks (1 point) on an extra credit question will be substantially more than earning the same amount of credit on other parts of the homework.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.


<b>Instructions</b>: For this homework we will be comparing our methods against a popular dataset of word similarities called Similarity-353. You need to first obtain this data set, which can be downloaded <a href="http://www.cs.technion.ac.il/~gabr/resources/data/wordsim353/wordsim353.zip">here</a>. The file we will be using is called *combined.tab*. Except for the header (which should be stripped out), the file is tab formated with the first two columns corresponding to two words, and the third column representing a human-annotated similarity between the two words.

Assume the file *combined.tab* is located <b>in the same folder as this notebook</b>. You should load this file into a Python dictionary (NOTE: in Python, tuples of strings, i.e. ("tiger","cat") can serve as the keys of dictionaries). This dataset contains many rare words: we need to filter this dataset in order for it to be better suited to the resources we will use in this assignment. So your first goal is to filter this dataset to generate a smaller test set where you will evaluate your word similarity methods.

The first filtering is based on document frequencies in the Brown corpus, in order to remove rare words. In this assignment, we will be treating the <i>paragraphs</i> of the Brown corpus as our "documents", you can iterate over them by using the `paras` method of the corpus reader. You should start by creating a Python list where each element of the list is a set containing the word <b>types</b> from a different paragraph of the Brown corpus: the words should be lower-cased and lemmatized before they are added to the set (keep it around, because you will need this list again later on). Then, using the information in this corpus, calculate document frequencies and remove from your test set any word pairs where at least one of the two words has a document frequency of less than 10 in this corpus. 

The second filtering is based on words with highly ambiguous senses and involves using the NLTK interface to WordNet. Here, you should remove any words which do not have a *single primary sense*. We define single primary sense here as either having only one sense (i.e. only one synset), or where the count (as provided by the WordNet `count()` method for the lemmas associated with a synset) of the most common sense is at least five and at least five times larger than the next most common sense. Also, you should remove any words where the primary sense is not a noun (this information is also in the synset). Store the synset corresponding to this primary sense in a dictionary for use in the next section. Given this definition, remove any word pairs from the test set where at least one of the words does not contain a single primary sense or if the single primary sense is not a noun.

When you have applied these two filtering steps, print out all the pairs in your filtered test set (if you have done this correctly, the total should be more than 10, but less than 50).

(1.5 marks)

In [1]:
from nltk.corpus import brown
from nltk.corpus import wordnet as wn
import nltk
import operator
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

# get the corpus: brown paragraphs in a list of sets
word_list = []
for paragraph in brown.paras():
    word_type = set()
    for sent in paragraph:
        for word in sent:
            w = word.lower()
            lemma = lemmatizer.lemmatize(w)
            word_type.add(lemma)
    word_list.append(word_type)

# return true if the two words both occur at least 10 times
def first_filter(tuple_key, word_list):
    count_0 = 0
    count_1 = 0
    for word in word_list:
        if tuple_key[0] in word:
            count_0 += 1
        if tuple_key[1] in word:
            count_1 += 1
        if count_0 >= 10 and count_1 >= 10:
            return True  # all greater than 10
    return False  # at least one less than 10

# return true if tuple key both have a single primary
def single_primary_sense(word):
    if len(wn.synsets(word)) == 1:
        if wn.synsets(word)[0].pos() == 'n':
            return True
        else:
            return False
    lemma_count = []
    for synset in wn.synsets(word):
        count = 0
        for lemma in synset.lemmas():
            if lemma.name() == word:
                count += lemma.count()
        lemma_count.append((count, synset.pos()))
    lemma_count.sort(key = operator.itemgetter(0))
    if lemma_count[-2][0] == 0 and lemma_count[-1][0] == 0:
        return False
    if lemma_count[-2][0] == 0 and lemma_count[-1][0] != 0:
        if (lemma_count[-1][0] >= 5) and (lemma_count[-1][1] == 'n'):
            return True
        else:
            return False
    if lemma_count[-1][0]/lemma_count[-2][0] >= 5:
        if lemma_count[-1][1] == 'n':
            return True
        else:
            return False
    else:
        return False
                
def second_filter(tuple_key):
    if single_primary_sense(tuple_key[0]) and single_primary_sense(tuple_key[1]):
        return True
    else:
        return False

f = open("combined.tab", "r")
lines = f.readlines()
length = len(lines)
dictionary = {}
for i in range(1, length):
    items = lines[i].split()
    key = (items[0], items[1])
    if first_filter(key, word_list) and second_filter(key):
        value = float(items[2])
        dictionary[key] = value

keys = dictionary.keys()
print(keys)

dict_keys([('car', 'automobile'), ('school', 'center'), ('brother', 'monk'), ('psychology', 'doctor'), ('doctor', 'personnel'), ('stock', 'egg'), ('hotel', 'reservation'), ('equipment', 'maker'), ('word', 'similarity'), ('computer', 'laboratory'), ('coast', 'hill'), ('coast', 'shore'), ('coast', 'forest'), ('psychology', 'mind'), ('psychology', 'science'), ('psychology', 'health'), ('baby', 'mother'), ('type', 'kind'), ('journey', 'voyage'), ('soap', 'opera'), ('luxury', 'car'), ('canyon', 'landscape'), ('century', 'year'), ('professor', 'doctor'), ('journey', 'car'), ('planet', 'people'), ('monk', 'slave')])


<b>Instructions</b>: Now you will create several dictionaries with similarity scores for pairs of words in your test set derived using the techniques discussed in class. The first of these is the Wu-Palmer scores derived from the hypernym relationships in WordNet, which you should calculate using the primary sense for each word derived above. You can use the built-in method included in the NLTK interface, you don't have to implement your own. When you're done,  print out the Python dictionary of word pair/similarity mappings. 

(0.5 marks)

In [2]:
# get the primary synset of the word
def primary_sense(word):
    lemma_count = []
    for synset in wn.synsets(word):
        count = 0
        for lemma in synset.lemmas():
            if lemma.name() == word:
                count += lemma.count()
        lemma_count.append((count, synset))
    lemma_count.sort(key = operator.itemgetter(0))
    return lemma_count[-1][1]

Wu_Palmer_scores_dict = {}
for key in dictionary.keys():
    synset_1 = primary_sense(key[0])
    synset_2 = primary_sense(key[1])
    Wu_Palmer_scores_dict[key] = synset_1.wup_similarity(synset_2)

print(Wu_Palmer_scores_dict)

{('word', 'similarity'): 0.3333333333333333, ('car', 'automobile'): 1.0, ('school', 'center'): 0.13333333333333333, ('brother', 'monk'): 0.5714285714285714, ('psychology', 'doctor'): 0.1111111111111111, ('doctor', 'personnel'): 0.13333333333333333, ('luxury', 'car'): 0.1111111111111111, ('hotel', 'reservation'): 0.375, ('equipment', 'maker'): 0.5, ('psychology', 'mind'): 0.5714285714285714, ('computer', 'laboratory'): 0.35294117647058826, ('coast', 'hill'): 0.6666666666666666, ('coast', 'shore'): 0.9090909090909091, ('coast', 'forest'): 0.16666666666666666, ('soap', 'opera'): 0.2222222222222222, ('psychology', 'science'): 0.9411764705882353, ('psychology', 'health'): 0.21052631578947367, ('baby', 'mother'): 0.5, ('type', 'kind'): 0.9473684210526315, ('journey', 'voyage'): 0.8571428571428571, ('stock', 'egg'): 0.11764705882352941, ('canyon', 'landscape'): 0.3333333333333333, ('century', 'year'): 0.8333333333333334, ('professor', 'doctor'): 0.5, ('journey', 'car'): 0.09523809523809523, (

**Instructions:** Next, you will calculate Positive PMI (PPMI) for your word pairs using statistics derived from the Brown: you should use the same set up as you did to calculate document frequency above: paragraphs as documents, lemmatized, lower-cased, and with term frequency information removed by conversion to Python sets. You need to use the basic method for calculating PPMI introduced in class (and also in the reading) which is appropriate for any possible definition of co-occurrence (here, appearing in the same paragraph), but you should only calculate PPMI for the words in your test set. You must avoid building the entire co-occurrence matrix, instead you should keeping track of the sums you need for the probabilities as you go along. When you have calculated PMI for all the pairs, your code should print out the Python dictionary of word-pair/PPMI-similarity mappings.

(1 mark)

In [7]:
from math import log

ppmi_dict = {}
for key in dictionary.keys():
    count_1 = 0
    count_2 = 0
    count_12 = 0
    total = 0
    # since each doc is a set, the words are unique in docs, we can count as follow
    for doc in word_list:
        # get how many unique word in the doc
        word_count_in_doc = len(doc)
        
        # if a target word occurs in this doc,
        # there are (word_count_in_doc - 1) combinations of two words which include the target word.
        if key[0] in doc:
            count_1 += word_count_in_doc - 1
        if key[1] in doc:
            count_2 += word_count_in_doc - 1
            
        # if both the two word occur in this doc, only one combination of this two words in this doc.
        if key[0] in doc and key[1] in doc:
            count_12 += 1
            
        # add the number of all possible combinations of two words in this doc
        total += word_count_in_doc * (word_count_in_doc - 1)

    # calculate ppmi and add into ppmi_dict 
    p1 = count_1/total
    p2 = count_2/total
    p12 = count_12/total
    if p12/(p1*p2) != 0:
        pmi = log(p12/(p1*p2), 2)
    else:
        pmi = 0
    if pmi < 0:
        ppmi = 0
    else:
        ppmi = pmi
    ppmi_dict[key] = ppmi

print(ppmi_dict)

{('word', 'similarity'): 0, ('car', 'automobile'): 2.0507593210421655, ('school', 'center'): 0, ('brother', 'monk'): 1.1188183995864585, ('psychology', 'doctor'): 1.4908868488505072, ('doctor', 'personnel'): 0.8020554021561166, ('luxury', 'car'): 0.6445089686902324, ('hotel', 'reservation'): 1.9652511271598834, ('equipment', 'maker'): 2.8501930465173153, ('psychology', 'mind'): 0.48396937228139286, ('computer', 'laboratory'): 0, ('coast', 'hill'): 0, ('coast', 'shore'): 3.0386645760073163, ('coast', 'forest'): 1.6814791563336242, ('soap', 'opera'): 2.8428019781528353, ('psychology', 'science'): 2.5024051628377033, ('psychology', 'health'): 0, ('baby', 'mother'): 1.773683996923688, ('type', 'kind'): 0, ('journey', 'voyage'): 0, ('stock', 'egg'): 0, ('canyon', 'landscape'): 0, ('century', 'year'): 0, ('professor', 'doctor'): 0, ('journey', 'car'): 0, ('planet', 'people'): 0, ('monk', 'slave'): 0}


**Instructions:** Next, you will derive similarity scores using the LSA method, i.e. apply SVD and truncate to get a dense vector and then use cosine similarity between the two vectors for each word pair. You can use the Distributed Semantics notebook as a starting point, but note that since you are interested here in word semantics, you will be constructing a matrix where the (non-sparse) rows correspond to words in the vocabulary, and the (sparse) columns correspond to the texts where they appear (this is the opposite of the notebook). Again, use the Brown corpus, in the same format as with PMI and document frequency. After you have a matrix in the correct format, use truncatedSVD in Sci-kit learn to produce dense vectors of length 500, and then use cosine similarity to produce similarities for your word pairs. Print out the corresponding Python dictionary.

(1 mark)

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import TruncatedSVD
from math import sqrt

def bow(doc):
    bow = {}
    for word in doc:
        bow[word] = bow.get(word, 0) + 1
    return bow

def cosine_similarity(x, y):
    length = len(x)
    numerator = 0
    denominator_x = 0
    denominator_y = 0
    for i in range(length):
        numerator += x[i] * y[i]
        denominator_x += x[i]**2
        denominator_y += y[i]**2
    return numerator/(sqrt(denominator_x)*sqrt(denominator_y))

# get features of documents and transmit to word-document matrix
documents = []
for doc in word_list:
    documents.append(bow(doc))
vectorizer = DictVectorizer()
matrix = vectorizer.fit_transform(documents).transpose()

# build the trancated matrix using svd with 500 components
names = vectorizer.get_feature_names()
trancate = TruncatedSVD(n_components = 500)
matrix = trancate.fit_transform(matrix)

# calculate the cosine similarity according to trancated matrix
num_of_words = len(matrix)
cos_sim_dict = {}
for key in dictionary.keys():
    index_1 = 0
    index_2 = 0
    for i in range(num_of_words):
        if key[0] == names[i]:
            index_1 = i
        if key[1] == names[i]:
            index_2 = i
    cos_sim_dict[key] = cosine_similarity(matrix[index_1], matrix[index_2])
    
print(cos_sim_dict)

{('word', 'similarity'): 0.0014829546682825433, ('car', 'automobile'): 0.34442168290402375, ('school', 'center'): 0.044483176454646133, ('brother', 'monk'): 0.11844725305511365, ('psychology', 'doctor'): 0.11544817081869356, ('doctor', 'personnel'): 0.0089542389809325086, ('luxury', 'car'): 0.080201950306108494, ('hotel', 'reservation'): 0.096873979703567203, ('equipment', 'maker'): 0.23675810802766978, ('psychology', 'mind'): 0.095282058426026578, ('computer', 'laboratory'): 0.14938453128253956, ('coast', 'hill'): 0.20785097671859701, ('coast', 'shore'): 0.40999365131663035, ('coast', 'forest'): 0.10445314807086661, ('soap', 'opera'): 0.042360310528688577, ('psychology', 'science'): 0.29202054223609897, ('psychology', 'health'): 0.02615530830180255, ('baby', 'mother'): 0.33691004370833522, ('type', 'kind'): 0.023867991345890464, ('journey', 'voyage'): 0.11632129652367136, ('stock', 'egg'): 0.13861873891099399, ('canyon', 'landscape'): 0.083506971638883334, ('century', 'year'): 0.06949

**Instructions:** Next, you will derive a similarity score from word2vec vectors, using the Gensim interface. Check the Gensim word2vec tutorial for details on the API: https://radimrehurek.com/gensim/models/word2vec.html. Again, you should use the Brown for this, but for word2vec you don't need to worry about paragraphs: feel free to train your model at the sentence level instead. Your vectors should have the same number of dimensions as LSA (500), and you need to run for 50 iterations. This may take a while (several minutes), but that's okay, you won't be marked based on the speed of this. You should extract the similarites you need directly from the Gensim model, put them in a Python dictionary, and print them out.

(0.5 mark)

In [5]:
# install gensim into notebook
# import sys
# !{sys.executable} -m pip install gensim

from gensim.models import Word2Vec

# train the model based on sentences level.
df = brown.sents()
model = Word2Vec(df, size=500, window=5, workers=4, iter=50)

# apply prediction using the training table
skip_gram_dict = {}
for key in dictionary.keys():
    skip_gram_dict[key] = model.wv.similarity(key[0], key[1])
    
print(skip_gram_dict)

C:\Users\msi\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


{('word', 'similarity'): -0.0070249244971005084, ('car', 'automobile'): 0.15790926389009358, ('school', 'center'): -0.036696250189935893, ('brother', 'monk'): 0.041508940517962334, ('psychology', 'doctor'): -0.007001343729141094, ('doctor', 'personnel'): -0.055851781846536025, ('luxury', 'car'): 0.11825175172149488, ('hotel', 'reservation'): 0.055948941691636278, ('equipment', 'maker'): 0.18857089530177348, ('psychology', 'mind'): 0.079953659064161187, ('computer', 'laboratory'): 0.18859362107034805, ('coast', 'hill'): 0.44379912376742281, ('coast', 'shore'): 0.43351491901210704, ('coast', 'forest'): 0.28590283299645936, ('soap', 'opera'): -0.0030967320907890686, ('psychology', 'science'): 0.34866524035796465, ('psychology', 'health'): 0.18494299301237402, ('baby', 'mother'): 0.22503419756527115, ('type', 'kind'): 0.28224183540504022, ('journey', 'voyage'): 0.46804117425541114, ('stock', 'egg'): 0.15080574013721559, ('canyon', 'landscape'): 0.1753655232863861, ('century', 'year'): 0.31


**Instructions:** Finally, you should compare all the similarities you've created to the gold standard you loaded and filtered in the first step. For this, you can use the Pearson correlation co-efficient (`pearsonr`), which is included in scipy (`scipy.stats`). Be careful converting your dictionaries to lists for this purpose, the data for the two datasets needs to be in the same order for correct comparison using correlation. Write a general function, then apply it to each of the similarity score dictionaries, and print out the result for each (be sure to label them!). Hint: All of the methods used here should be markedly above 0, but also far from 1 (perfect correlation); if you're not getting reasonable results, go back and check your code for bugs!  

(0.5 mark)


In [8]:
import scipy
gold_standard = []
similarity_1 = []
similarity_2 = []
similarity_3 = []
similarity_4 = []

# get each similarity score in correct sequence.
for key in dictionary.keys():
    gold_standard.append(dictionary[key])
    similarity_1.append(Wu_Palmer_scores_dict[key])
    similarity_2.append(ppmi_dict[key])
    similarity_3.append(cos_sim_dict[key])
    similarity_4.append(skip_gram_dict[key])

# compare the similarities based on pearsonr correlation.
print("Correlation (Wu_Palmer):   " + str(scipy.stats.pearsonr(similarity_1, gold_standard)[0]))
print("Correlation (PPMI):        " + str(scipy.stats.pearsonr(similarity_2, gold_standard)[0]))
print("Correlation (LSA):         " + str(scipy.stats.pearsonr(similarity_3, gold_standard)[0]))
print("Correlation (skip_gram):   " + str(scipy.stats.pearsonr(similarity_4, gold_standard)[0]))

Correlation (Wu_Palmer):   0.456692740637
Correlation (PPMI):        0.312401071068
Correlation (LSA):         0.368243821253
Correlation (skip_gram):   0.343077101305


## Bonus: Improving the correlation

**Instructions:** To get the extra credit, you should try to get a noticeably (at least 0.05) higher correlation than that of the highest correlation that you found in the main assignment. To accomplish this, you can use any resources included or accessible via NLTK EXCEPT the Google word2vec vectors. Half of your mark here will be based on the thought and effort involved in your solution (if you tried something sensible that didn't work, show us), the other half will be based on the actual performance increase, which you should demonstrate. For full credit, be sure to explain exactly what you are doing.

(1 bonus mark) 

In [9]:
"""
Explanation:

From the 4 pearsonr correlation results above, we can see the lexical semantics performed better than distributional semantics,
So we can try to improve the lexical semantics approach to improve the correlation. Instead of using Wu-Palmer, we tryed 
Lin distance which give a larger correlation than Wu-Palmer (more than 0.05). 

Lin distance estimate the similarity between two words based on information content (we used ic-semcor.dat in wordnet_ic).
The correlation is improved (more than 0,05) shown as below.

WordNet Interface Reference:  http://www.nltk.org/howto/wordnet.html
"""


from nltk.corpus import wordnet_ic
semcor_ic = wordnet_ic.ic('ic-semcor.dat')

lin = []
for key in dictionary.keys():
    synset_1 = primary_sense(key[0])
    synset_2 = primary_sense(key[1])
    lin.append(synset_1.lin_similarity(synset_2, semcor_ic))
    
print("Correlation (Lin distance):   " + str(scipy.stats.pearsonr(lin, gold_standard)[0]))

Correlation (Lin distance):   0.535496645448


## A final word

Normally, we would not use a corpus as small as the Brown for the purposes of building distributional word vectors. Also, note that filtering our test set to just words we are likely to do well on would typically be considered cheating.